In [11]:
from keras.models import Sequential
from keras.layers import Dense, Input, Concatenate
from keras import Model
from tensorflow import keras

In [12]:
import os
import pandas as pd
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
opt = keras.optimizers.Adam()

In [15]:
type_path = "sample_data/test_data"
graph_data = { }
game_play_data = { }
hash_data = {}
environments = {}

In [16]:
level_cap = 5
configuration_data = {}

for filename in os.listdir(type_path):
    file_path = os.path.join(type_path, filename)
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        if "level_selection" in file_path:
            if filename not in hash_data:
                hash_data[filename] = []
                environments[filename] = {}
            for i in range(len(df)):
                battle_environment = df.loc[i, 'BattleEnvironment']
                battle_id = df.loc[i, 'ID']
                if battle_environment not in environments:
                    environments[filename][battle_environment] = [0, 0, 0, 0, 0]
                if battle_id not in hash_data:
                    level = df.loc[i, 'Level']
                    environments[filename][battle_environment][level] = environments[filename][battle_environment][level] + 1
                    hash_data[filename].append(battle_id)
        elif "level_configuration" in filename:
            for i in range(len(df)):
              config = df.iloc[i].values.tolist()
              print(config)
              configuration_data[config[0]] = config[1:]
              configuration_data["1"] = config[1:]
        else:
            if filename not in game_play_data:
                game_play_data[filename] = {
                    "Brawler": {},
                    "Mage": {},
                    "Swordsman": {}
                }
                for key in game_play_data[filename]:
                    game_play_data[filename][key] = [[], [], [], [], []]
                graph_data[filename] = {}
            adventurer_type = filename.split('_')[0]
            for i in range(len(df)):
                level = df.loc[i, 'Level']
                max_steps = df.loc[i, 'MaxCount']
                game_play_data[filename][adventurer_type][level-1].append(max_steps)
            for key in game_play_data[filename]:
                graph_data[filename][key] = [[], [], [], [], []]
                for x in range(level_cap):
                    if len(game_play_data[filename][key][x]) > 0:
                        graph_data[filename][key][x] = mean(game_play_data[filename][key][x])

['02/08/2022 00:32:31', 0.2001211, 0.1932651, 0.2002864]


In [17]:
print(configuration_data)
print(game_play_data)

{'02/08/2022 00:32:31': [0.2001211, 0.1932651, 0.2002864], '1': [0.2001211, 0.1932651, 0.2002864]}
{'Brawler_1_02_08_2022_00_49.csv': {'Brawler': [[494, 1047, 1453, 9190, 11785, 12378, 14560, 20776, 20776, 378, 2840], [8829, 15548, 18060, 19795, 19863, 20417, 21139, 21762, 27576, 3054, 4660], [10534, 22181, 26814, 30327, 35099, 41009, 46389, 56164, 58369, 7926, 15190], [36723, 46389, 51863, 63547, 76626, 79677, 2328, 3964, 4257, 31775], [77339, 0, 18269, 0, 19783, 0]], 'Mage': [[], [], [], [], []], 'Swordsman': [[], [], [], [], []]}, 'Swordsman_1_02_08_2022_00_49.csv': {'Brawler': [[], [], [], [], []], 'Mage': [[], [], [], [], []], 'Swordsman': [[494, 1453, 2421, 6915, 9918, 11785, 13652, 2252, 9840, 4520], [1220, 1220, 4259, 12314, 13652, 14215, 15112, 21430, 21430, 28681, 5763, 480, 6785, 2015, 15286, 10027, 14770], [14560, 14972, 20417, 22201, 25807, 28365, 28831, 35099, 37490, 42255, 66534, 13000, 13281, 13161], [29660, 34003, 37490, 41312, 48011, 49147, 50170, 64643, 68249, 77007,

In [18]:
print(config)

['02/08/2022 00:32:31', 0.2001211, 0.1932651, 0.2002864]


In [19]:
reviews = { "02/08/2022 00:32:31": [0, 0, 1], "Test2": [0, 0, 1] }
from sklearn.model_selection import train_test_split
y = list(reviews.keys())
labels = []
labeled_config = []
for key in y:
  if key in configuration_data:
    data = configuration_data[key]
    labeled_config.append(data)
    labels.append(reviews[key])

unlabeled_data = []
for key in configuration_data:
  if key not in y:
    data = configuration_data[key]
    unlabeled_data.append(data)

In [20]:
print(labeled_config)
print(config[0])
config = [config, config]
print(labels)
labels.append(labels[0])
print(unlabeled_data)

[[0.2001211, 0.1932651, 0.2002864]]
02/08/2022 00:32:31
[[0, 0, 1]]
[[0.2001211, 0.1932651, 0.2002864]]


In [33]:
label_data = [labeled_config[0], labeled_config[0]]
print(labels)

[[0, 0, 1], [0, 0, 1]]


In [35]:
train_data, train_labels, val_data, val_labels = train_test_split(label_data, labels, test_size=0.1)

In [23]:
# Helper function for merging new history objects with older ones
def append_history(losses, val_losses, accuracy, val_accuracy, history):
    losses = losses + history.history["loss"]
    val_losses = val_losses + history.history["val_loss"]
    accuracy = accuracy + history.history["categorical_accuracy"]
    val_accuracy = val_accuracy + history.history["val_categorical_accuracy"]
    return losses, val_losses, accuracy, val_accuracy


# Plotter function
def plot_history(losses, val_losses, accuracies, val_accuracies):
    plt.plot(losses)
    plt.plot(val_losses)
    plt.legend(["train_loss", "val_loss"])
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.plot(accuracies)
    plt.plot(val_accuracies)
    plt.legend(["train_accuracy", "val_accuracy"])
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()

In [24]:
import tensorflow as tf

In [28]:
def make_model():
  x = Input(shape=(3,), name="Game Play Data")

  g = Dense(500, activation='relu')(x)
  g = Dense(100, activation='relu')(g)
  g = Dense(50, activation='relu')(g)
  g = Dense(3, activation='softmax')(g)
  model = Model(inputs=x, outputs=g)
  return model

In [30]:
print(train_data)

[['02/08/2022 00:32:31', 0.2001211, 0.1932651, 0.2002864]]


In [36]:
num_iterations=3
sampling_size=5000
# inspired from this https://keras.io/examples/nlp/active_learning_review_classification/
model = make_model()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=[
          keras.metrics.CategoricalAccuracy(),
          keras.metrics.FalseNegatives(),
          keras.metrics.FalsePositives(),
      ],)
losses, val_losses, accuracies, val_accuracies = [], [], [], []

# Defining checkpoints.
# The checkpoint callback is reused throughout the training since it only saves the best overall model.
checkpoint = keras.callbacks.ModelCheckpoint(
    "AL_Model.h5", save_best_only=True, verbose=1
)
# Here, patience is set to 4. This can be set higher if desired.
early_stopping = keras.callbacks.EarlyStopping(patience=4, verbose=1)

print(f"Starting to train with {len(train_data)} samples")
# Initial fit with a small subset of the training set
history = model.fit(
    x=train_data,
    y=train_labels,
    epochs=20,
    validation_data=(val_data, val_labels),
    callbacks=[checkpoint, early_stopping],
)

# Appending history
losses, val_losses, accuracies, val_accuracies = append_history(
    losses, val_losses, accuracies, val_accuracies, history
)

Starting to train with 1 samples
Epoch 1/20
1/1 [==============================] - ETA: 0s - loss: 0.6522 - categorical_accuracy: 1.0000 - false_negatives_2: 3.0000 - false_positives_2: 0.0000e+00
Epoch 1: val_loss improved from inf to 1.05867, saving model to AL_Model.h5
1/1 [==============================] - 2s 2s/step - loss: 0.6522 - categorical_accuracy: 1.0000 - false_negatives_2: 3.0000 - false_positives_2: 0.0000e+00 - val_loss: 1.0587 - val_categorical_accuracy: 1.0000 - val_false_negatives_2: 1.0000 - val_false_positives_2: 0.0000e+00
Epoch 2/20
1/1 [==============================] - ETA: 0s - loss: 0.6522 - categorical_accuracy: 1.0000 - false_negatives_2: 3.0000 - false_positives_2: 0.0000e+00
Epoch 2: val_loss improved from 1.05867 to 1.05525, saving model to AL_Model.h5
1/1 [==============================] - 0s 81ms/step - loss: 0.6522 - categorical_accuracy: 1.0000 - false_negatives_2: 3.0000 - false_positives_2: 0.0000e+00 - val_loss: 1.0552 - val_categorical_accuracy: 

In [37]:
used = []
for iteration in range(num_iterations):
  # Getting predictions from previously trained model
  predictions = model.predict(unlabeled_data)
  length = min(len(predictions), 10)
  minValue = 1
  for x in range(length):
    if x not in used:
      predict = predictions[x]
      maxValue = max(predict)
      if maxValue < minValue:
        minValue = maxValue
  if minValue != 1:
    used.append()

  continue
  

  # We recompile the model to reset the optimizer states and retrain the model
  model.compile(
      loss="categorical_crossentropy",
      optimizer="rmsprop",
      metrics=[
          keras.metrics.CategoricalAccuracy(),
          keras.metrics.FalseNegatives(),
          keras.metrics.FalsePositives(),
      ],
  )
  history = model.fit(
      x=train_data,
      y=train_labels,
      validation_data=(val_data, val_labels),
      epochs=20,
      callbacks=[
          checkpoint,
          keras.callbacks.EarlyStopping(patience=4, verbose=1),
      ],
  )

  # Appending the history
  losses, val_losses, accuracies, val_accuracies = append_history(
      losses, val_losses, accuracies, val_accuracies, history
  )

  # Loading the best model from this training loop
  model = keras.models.load_model("AL_Model.h5")

In [ ]:
# Plotting the overall history and evaluating the final model
  plot_history(losses, val_losses, accuracies, val_accuracies)
  print("-" * 100)
  print(
      "Test set evaluation: ",
      model.evaluate(test_dataset, verbose=0, return_dict=True),
  )
  print("-" * 100)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)